In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import json
with open('Dataset.json') as f:
    training_data = json.load(f)

In [ ]:
# training_data

In [ ]:
len(training_data)

28900

In [ ]:
!pip install rouge-score
!pip install nltk
!pip install textstat

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=300520cb5c6faddd2a54bd7ba89a48bdd2b4a7503138d111bdebccc56621f807
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.0 MB/s eta 0:00:00


In [ ]:
!pip install rouge

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# استيراد المكتبات اللازمة للنظام
import os  # لإدارة الملفات والمسارات
import torch  # مكتبة تعلم عميق لدعم PyTorch
import numpy as np  # للتعامل مع المصفوفات والعمليات الرياضية
from typing import List, Dict, Optional  # تحديد أنواع البيانات للوظائف
from transformers import (  # استيراد مكونات مكتبة Hugging Face
    AutoModelForCausalLM,  # نموذج لغوي للتنبؤ بالنصوص
    AutoTokenizer,  # مُحلل النصوص لتحويل النصوص إلى رموز
    Trainer,  # أداة تدريب للنماذج
    TrainingArguments,  # إعدادات التدريب
    TrainerCallback  # ردود فعل مخصصة خلال التدريب
)
from datasets import Dataset, load_dataset  # إنشاء وتحميل مجموعات البيانات
from sklearn.model_selection import train_test_split  # تقسيم البيانات للتدريب والاختبار
from sklearn.metrics import (  # حساب مقاييس الأداء
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)

# التحقق من تحميل موارد مكتبة NLTK
import nltk
try:
    nltk.data.find('tokenizers/punkt')  # التحقق من وجود القاموس
except LookupError:
    print("Downloading NLTK 'punkt' resource...")  # إذا لم تكن موجودة، تنزيلها
    nltk.download('punkt', quiet=True)

# مكتبات إضافية للتقييم
import rouge  # مكتبة لحساب دقة النصوص (ROUGE)
from nltk.translate.bleu_score import sentence_bleu  # حساب مقياس BLEU
from nltk.tokenize import word_tokenize  # تقسيم النصوص إلى كلمات

# تعريف فئة ModelFineTuner لضبط النماذج اللغوية
class ModelFineTuner:
    def __init__(self, model_name: str = "EleutherAI/gpt-neo-125M", freeze_percentage: float = 0.8, device: Optional[str] = None):
        """
        تهيئة النموذج اللغوي والمحلل النصي مع إمكانية تخصيص اسم النموذج ونسبة التجميد والجهاز المستخدم.
        """
        # تحديد الجهاز (GPU أو CPU)
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        # تحميل النموذج اللغوي والمحلل النصي
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # إضافة رمز التوسيع إذا لم يكن موجودًا
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model.config.pad_token_id = self.model.config.eos_token_id

        # نقل النموذج إلى الجهاز
        self.model = self.model.to(self.device)

        # تجميد بعض طبقات النموذج
        self._freeze_model_layers(freeze_percentage)

        # إضافة متغيرات لتتبع المحادثة
        self.conversation_history = []
        self.max_history_length = 10  # عدد الرسائل السابقة التي سيتم تتبعها
        self.max_tokens = 2048  # أقصى عدد للمدخلات حسب وثائق GPT-Neo
        self.model.config.max_length = 2048
        self.model.config.output_attentions = True
        self.model.config.use_cache = True

    def _freeze_model_layers(self, freeze_percentage):
        """
        تجميد نسبة معينة من طبقات النموذج.
        """
        total_layers = len(self.model.transformer.h)  # إجمالي عدد الطبقات
        freeze_layers = int(total_layers * freeze_percentage)  # حساب عدد الطبقات المجمدة

        for i, layer in enumerate(self.model.transformer.h):
            if i < freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False  # إيقاف تحديث الأوزان
            else:
                for param in layer.parameters():
                    param.requires_grad = True  # السماح بتحديث الأوزان

    def prepare_dataset(self, data: List[Dict[str, str]], test_size: float = 0.1, max_length: int = 128):
        """
        تجهيز البيانات للتدريب مع تقسيمها إلى تدريب واختبار.
        """
        # تقسيم البيانات إلى مجموعتي تدريب واختبار
        train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)

        # وظيفة للمعالجة المسبقة لكل مثال
        def preprocess(example):
            input_text = f"{example['question']} <|endoftext|> {example['answer']}"
            tokens = self.tokenizer(input_text, truncation=True, padding="max_length", max_length=max_length)
            tokens["labels"] = tokens["input_ids"].copy()  # إعداد النصوص للإخراج
            return tokens

        # تحويل البيانات إلى مجموعات بيانات Dataset
        train_dataset = Dataset.from_list(train_data).map(preprocess)
        test_dataset = Dataset.from_list(test_data).map(preprocess)

        return train_dataset, test_dataset

    def train(self, train_dataset, validation_dataset=None, output_dir: str = "./results",
              num_train_epochs: int = 3, per_device_train_batch_size: int = 4):
        """
        تحسين عملية تدريب النموذج مع إضافة تقنيات متقدمة للتدريب والتحقق
        """
        # عدد الخطوات في الجولة الواحدة
        steps_per_epoch = len(train_dataset) // per_device_train_batch_size

        # تعديل eval_steps ليكون مضاعفاً صحيحاً لـ save_steps
        save_steps = 1024
        eval_steps = save_steps  # جعل eval_steps مساوياً لـ save_steps لضمان التوافق

        # إضافة جدولة معدل التعلم المتكيف
        def lr_scheduler(current_step: int, num_warmup_steps: int, num_training_steps: int):
            if current_step < num_warmup_steps:
                return float(current_step) / float(max(1, num_warmup_steps))
            return max(0.0, float(num_training_steps - current_step) /
                      float(max(1, num_training_steps - num_warmup_steps)))

        # إعدادات التدريب المحسنة
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_train_batch_size,
            warmup_steps=100,  # خطوات التهيئة
            weight_decay=0.01,  # تنظيم الأوزان
            logging_dir=f"{output_dir}/logs",
            logging_steps=10,
            save_strategy="steps",
            save_steps=save_steps,
          #  evaluation_strategy="steps" if validation_dataset else "no",
            eval_steps=eval_steps if validation_dataset else None,
            load_best_model_at_end=True if validation_dataset else False,
            metric_for_best_model="eval_loss" if validation_dataset else None,
            greater_is_better=False,
            learning_rate=2e-5,
            lr_scheduler_type="cosine_with_restarts",
            gradient_accumulation_steps=4,
            fp16=torch.cuda.is_available(),
            report_to="none"
        )

        # إضافة مراقب مخصص للتدريب
        class CustomCallback(TrainerCallback):
            def __init__(self):
                self.best_loss = float('inf')
                self.no_improve_count = 0
                self.early_stop_patience = 3

            def on_evaluate(self, args, state, control, metrics, **kwargs):
                current_loss = metrics.get("eval_loss", float('inf'))
                if current_loss < self.best_loss:
                    self.best_loss = current_loss
                    self.no_improve_count = 0
                else:
                    self.no_improve_count += 1

                if self.no_improve_count >= self.early_stop_patience:
                    control.should_training_stop = True
                    print("\nEarly stopping triggered!")

            def on_log(self, args, state, control, logs, **kwargs):
                if state.is_local_process_zero and 'loss' in logs:
                    current_loss = logs['loss']
                    print(f"\rCurrent Loss: {current_loss:.4f}", end="")

        # تحسين تجهيز النموذج للتدريب
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            # حساب مقاييس متعددة
            return {
                'perplexity': torch.exp(torch.tensor(predictions.mean())).item(),
                'loss': predictions.mean().item()
            }

        # إنشاء المدرب مع الإعدادات المحسنة
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=validation_dataset,
            compute_metrics=compute_metrics if validation_dataset else None,
            callbacks=[CustomCallback()]
        )

        # تطبيق تقنية gradient clipping
        trainer.args.max_grad_norm = 1.0

        # استخدام تقنية mixed precision training
        if torch.cuda.is_available():
            trainer.args.fp16 = True
            trainer.args.fp16_opt_level = "O1"

        # بدء التدريب
        trainer_output = trainer.train()

        # حفظ النموذج والتوكنايزر
        trainer.save_model(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        return trainer_output

    def evaluate_model(self, test_dataset, output_dir: str = "./results"):
        """
        تقييم أداء النموذج على بيانات الاختبار باستخدام عدة مقاييس.

        :param test_dataset: مجموعة بيانات الاختبار.
        :param output_dir: المجلد لحفظ نتائج التقييم.
        :return: قاموس يحتوي على مقاييس الأداء.
        """
        # تجهيز بيانات الاختبار
        actual_questions = [item['question'] for item in test_dataset]  # استخراج الأسئلة الأصلية.
        actual_answers = [item['answer'] for item in test_dataset]  # استخراج الإجابات الأصلية.

        # توليد إجابات للنموذج لكل سؤال
        predicted_answers = []
        for question in actual_questions:
            predicted_answer = self.generate_answer(
                question,  # السؤال المُدخل
                max_length=128,  # الطول الأقصى للإجابة
                num_return_sequences=1  # عدد الإجابات المطلوبة
            )
            predicted_answers.append(predicted_answer)  # تخزين الإجابة المولدة.

        # حساب مقاييس التقييم
        evaluation_metrics = {}

        # 1. حساب مقاييس ROUGE
        rouge_scorer = rouge.Rouge()  # إنشاء كائن ROUGE لحساب المقاييس.
        rouge_scores = {
            'rouge1': [],  # قائمة لتخزين قيم ROUGE-1.
            'rouge2': [],  # قائمة لتخزين قيم ROUGE-2.
            'rougeL': []  # قائمة لتخزين قيم ROUGE-L.
        }

        for ref, pred in zip(actual_answers, predicted_answers):
            # حساب ROUGE لكل زوج من الإجابات الأصلية والمتوقعة.
            rouge_result = rouge_scorer.get_scores(pred, ref)[0]
            rouge_scores['rouge1'].append(rouge_result['rouge-1']['f'])
            rouge_scores['rouge2'].append(rouge_result['rouge-2']['f'])
            rouge_scores['rougeL'].append(rouge_result['rouge-l']['f'])

        # حساب متوسط قيم ROUGE
        evaluation_metrics['rouge1_mean'] = np.mean(rouge_scores['rouge1'])
        evaluation_metrics['rouge2_mean'] = np.mean(rouge_scores['rouge2'])
        evaluation_metrics['rougeL_mean'] = np.mean(rouge_scores['rougeL'])

        # 2. حساب مقياس BLEU
        bleu_scores = []
        for ref, pred in zip(actual_answers, predicted_answers):
            # إعداد المرجعية (الإجابة الأصلية) والإجابة المُولدة.
            reference = [word_tokenize(ref.lower())]  # تقسيم النص إلى كلمات.
            candidate = word_tokenize(pred.lower())  # تقسيم النص المُولد إلى كلمات.
            bleu_scores.append(sentence_bleu(reference, candidate))  # حساب BLEU.

        evaluation_metrics['bleu_score'] = np.mean(bleu_scores)  # متوسط BLEU.

        # 3. حساب متوسط طول الإجابات
        evaluation_metrics['avg_pred_length'] = np.mean([len(ans.split()) for ans in predicted_answers])
        evaluation_metrics['avg_actual_length'] = np.mean([len(ans.split()) for ans in actual_answers])

        # 4. حساب نسبة التطابق الجزئي
        partial_match_count = sum(
            actual.lower() in pred.lower() or pred.lower() in actual.lower()
            for actual, pred in zip(actual_answers, predicted_answers)
        )
        evaluation_metrics['partial_match_ratio'] = partial_match_count / len(actual_answers)

        # 5. حساب التباين والانحراف المعياري لطول الإجابات
        evaluation_metrics['answer_variance'] = np.var([len(ans) for ans in predicted_answers])
        evaluation_metrics['answer_std_dev'] = np.std([len(ans) for ans in predicted_answers])

        # طباعة وحفظ النتائج
        print("نتائج تقييم النموذج:")
        for metric, value in evaluation_metrics.items():
            print(f"{metric}: {value}")

        # حفظ النتائج في ملف JSON
        import json
        os.makedirs(output_dir, exist_ok=True)  # إنشاء المجلد إذا لم يكن موجودًا.
        with open(os.path.join(output_dir, "evaluation_metrics.json"), "w", encoding="utf-8") as f:
            json.dump(evaluation_metrics, f, ensure_ascii=False, indent=4)

        return evaluation_metrics
    def add_to_history(self, question: str, answer: str):
        """
        إضافة محادثة جديدة إلى السجل
        """
        self.conversation_history.append({"question": question, "answer": answer})
        # الاحتفاظ فقط بآخر max_history_length من المحادثات
        if len(self.conversation_history) > self.max_history_length:
            self.conversation_history.pop(0)
    def format_conversation_history(self) -> str:
        """
        تنسيق سجل المحادثات مع تحسين إدارة السياق
        """
        formatted_history = ""
        history_length = len(self.conversation_history)

        if history_length == 0:
            return formatted_history

        # حساب أهمية كل محادثة
        conversation_weights = []
        for i, conv in enumerate(self.conversation_history):
            # عوامل الأهمية:
            recency_weight = (i + 1) / history_length  # الأحدث له وزن أكبر
            length_weight = min(len(conv['question'].split()) / 50, 1.0)  # طول السؤال
            # يمكن إضافة المزيد من العوامل هنا

            total_weight = (recency_weight * 0.7) + (length_weight * 0.3)
            conversation_weights.append((i, total_weight))

        # ترتيب المحادثات حسب الأهمية
        conversation_weights.sort(key=lambda x: x[1], reverse=True)

        # اختيار أهم 5 محادثات
        selected_indices = [i for i, _ in conversation_weights[:5]]
        selected_indices.sort()  # إعادة ترتيبها زمنياً

        # تشكيل السياق المحسن
        formatted_history = "Important context from our previous conversation:\n\n"
        for idx in selected_indices:
            conv = self.conversation_history[idx]
            formatted_history += (
                f"Topic {idx + 1}:\n"
                f"User asked: {conv['question']}\n"
                f"Response: {conv['answer']}\n\n"
            )

        return formatted_history

    def generate_answer(self, question: str, max_length: int = 512, num_return_sequences: int = 1, use_history: bool = True):
        """
        توليد إجابة مباشرة من خلال توجيه أفضل للنموذج
        """
        # تعريف الحد الأدنى والأقصى المطلوب لطول الإجابة
        MIN_RESPONSE_LENGTH = 100
        MAX_RESPONSE_LENGTH = 300

        if use_history and self.conversation_history:
            # تحسين استخدام السياق السابق
            recent_conversations = self.conversation_history[-1:]
            conversation_context = "\n".join([
                f"Previous Q: {conv['question']}\nA: {conv['answer']}"
                for conv in recent_conversations
            ])

            # تحسين صياغة الـ prompt لتوجيه النموذج بشكل أكثر فعالية
            prompt = (
                f"Context from recent conversation:\n{conversation_context}\n\n"
                f"New question: {question}\n\n"
                "Direct answer: "
            )
        else:
            prompt = (
                f"Question: {question}\n\n"
                "Direct answer: "
            )

        # تحسين معلمات التوليد للحصول على إجابات أكثر تماسكاً ومباشرة
        generation_params = {
            'max_length': min(self.max_tokens, max_length),
            'num_return_sequences': num_return_sequences,
            'pad_token_id': self.tokenizer.eos_token_id,
            'do_sample': True,
            'top_k': 40,         # زيادة للسماح بمزيد من التنوع
            'top_p': 0.92,       # زيادة طفيفة للسماح بمزيد من الإبداع مع الحفاظ على التماسك
            'temperature': 0.7,   # درجة حرارة معتدلة للتوازن بين الإبداع والتماسك
            'no_repeat_ngram_size': 3,
            'repetition_penalty': 1.2,
            'length_penalty': 1.0,
            'num_beams': 4,      # زيادة عدد الشعاع للحصول على إجابات أكثر ترابطاً
            'early_stopping': True,
            'min_length': MIN_RESPONSE_LENGTH
        }

        # تشفير النص
        encoded = self.tokenizer(
            prompt,
            truncation=True,
            max_length=self.max_tokens - MAX_RESPONSE_LENGTH,
            padding=True,
            return_tensors="pt",
            return_attention_mask=True
        ).to(self.device)

        # محاولة توليد الإجابة
        for attempt in range(2):
            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids=encoded['input_ids'],
                    attention_mask=encoded['attention_mask'],
                    **generation_params
                )

            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # استخراج الإجابة
            if "Direct answer:" in generated_text:
                answer = generated_text.split("Direct answer:")[-1].strip()
            else:
                answer = generated_text.replace(prompt, "").strip()

            word_count = len(answer.split())

            # التحقق من طول الإجابة
            if MIN_RESPONSE_LENGTH <= word_count <= MAX_RESPONSE_LENGTH:
                break

            # تعديل المعلمات للمحاولة الثانية إذا كان الطول غير مناسب
            if attempt == 0:
                if word_count < MIN_RESPONSE_LENGTH:
                    generation_params['min_length'] = MIN_RESPONSE_LENGTH
                    generation_params['length_penalty'] = 1.5
                    generation_params['temperature'] = 0.8
                else:  # طويلة جداً
                    generation_params['max_length'] = max_length // 2
                    generation_params['length_penalty'] = 0.8
                    generation_params['temperature'] = 0.7

        # إضافة إلى السجل فقط إذا كانت الإجابة مقبولة
        if word_count >= MIN_RESPONSE_LENGTH:
            self.add_to_history(question, answer)
            self.update_response_statistics(word_count)

        return answer

    def update_response_statistics(self, word_count: int):
        """
        تحديث إحصائيات الإجابات للمساعدة في ضبط طول الإجابات المستقبلية
        """
        if not hasattr(self, 'response_lengths'):
            self.response_lengths = []

        self.response_lengths.append(word_count)
        if len(self.response_lengths) > 10:  # الاحتفاظ بآخر 10 إجابات فقط
            self.response_lengths.pop(0)

In [ ]:
# مثال على الاستخدام
fine_tuner = ModelFineTuner(
    model_name="EleutherAI/gpt-neo-125M",
    freeze_percentage=0.95,
    device="cuda"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [ ]:
# تحضير مجموعة البيانات مع التقسيم
train_dataset, test_dataset = fine_tuner.prepare_dataset(training_data)

Map:   0%|          | 0/26010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2890 [00:00<?, ? examples/s]

In [ ]:
# تدريب النموذج
trained_model = fine_tuner.train(train_dataset)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
10,7.761300
20,7.870000
30,7.551600
40,7.159000
50,6.098600
60,5.308800
70,4.081600
80,2.859800
90,2.026700
100,1.675000


Current Loss: 0.6794

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 2048}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Current Loss: 0.6072

# **GUI (User Interface)**

In [ ]:
!pip install googletrans diffusers nest_asyncio

In [ ]:
html = """
<!DOCTYPE html>
<html>
<head>
    <title>AI Tourism Assistant - Saudi Arabia</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css" rel="stylesheet">
    <style>
        body {
            font-family: 'Tajawal', sans-serif;
            background-color: #f8f9fa;
            min-height: 100vh;
            background-image: url('https://via.placeholder.com/1500x1000');
            background-size: cover;
            background-position: center;
        }
        .chat-container {
            max-width: 1200px;
            margin: 2rem auto;
            background: rgba(255, 255, 255, 0.9);
            border-radius: 15px;
            box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
            overflow: hidden;
            backdrop-filter: blur(10px);
        }
        .chat-messages {
            height: 500px;
            overflow-y: auto;
            padding: 1.5rem;
            background: rgba(255, 255, 255, 0.8);
        }
        .message {
            margin-bottom: 1.5rem;
            opacity: 0;
            transform: translateY(20px);
            animation: fadeIn 0.3s ease forwards;
        }
        @keyframes fadeIn {
            to { opacity: 1; transform: translateY(0); }
        }
        .user-message { display: flex; justify-content: flex-end; }
        .bot-message { display: flex; justify-content: flex-start; }
        .message-content {
            max-width: 80%;
            padding: 1rem;
            border-radius: 15px;
        }
        .user-message .message-content {
            background: #218838;
            color: white;
            border-bottom-right-radius: 5px;
        }
        .bot-message .message-content {
            background: #f8f9fa;
            border: 1px solid #e9ecef;
            border-bottom-left-radius: 5px;
            color: #333;
        }
        .input-container {
            padding: 1.5rem;
            background: rgba(255, 255, 255, 0.9);
            border-top: 1px solid #e9ecef;
        }
        .input-group {
            background: white;
            border-radius: 25px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
        .form-control { border: none; padding: 0.75rem 1.5rem; font-size: 1rem; }
        .form-control:focus { box-shadow: none; }
        .send-button {
            padding: 0.75rem 1.5rem;
            background: #218838;
            border: none;
            color: white;
            transition: all 0.3s ease;
        }
        .send-button:hover { background: #1e7e34; }
        .typing-indicator {
            display: none;
            padding: 1rem;
            background: #f8f9fa;
            border-radius: 15px;
            margin-bottom: 1rem;
        }
        .typing-dot {
            display: inline-block;
            width: 8px;
            height: 8px;
            margin-right: 3px;
            background: #218838;
            border-radius: 50%;
            animation: typing 1s infinite;
        }
        .typing-dot:nth-child(2) { animation-delay: 0.2s; }
        .typing-dot:nth-child(3) { animation-delay: 0.4s; }
        @keyframes typing {
            0%, 100% { transform: translateY(0); }
            50% { transform: translateY(-5px); }
        }
        .system-message {
            text-align: center;
            color: #6c757d;
            margin: 1rem 0;
            font-style: italic;
        }
        .avatar {
            width: 40px;
            height: 40px;
            border-radius: 50%;
            margin: 0 10px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 1.2rem;
        }
        .bot-avatar { background: #e9ecef; color: #218838; }
        .user-avatar { background: #218838; color: white; }
        .chat-header {
            background: #218838;
            color: white;
            padding: 1rem;
            text-align: center;
            font-size: 1.5rem;
            font-weight: bold;
        }
        .chat-header i { margin-right: 10px; }
        .translation-indicator {
            font-size: 0.75rem;
            color: #6c757d;
            margin-top: 5px;
            font-style: italic;
        }
        .generated-image {
            max-width: 100%;
            border-radius: 10px;
            margin-top: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        .image-option { display: flex; align-items: center; margin-top: 10px; }
        .image-checkbox { margin-right: 10px; }
        .image-label { font-size: 0.9rem; color: #6c757d; }
        .image-generating {
            font-size: 0.9rem;
            margin-top: 10px;
            color: #218838;
            display: flex;
            align-items: center;
        }
        .progress-container {
            width: 100%;
            background-color: #e9ecef;
            border-radius: 10px;
            margin-top: 5px;
        }
        .progress-bar {
            height: 8px;
            background-color: #218838;
            width: 0%;
            transition: width 0.5s;
        }
        .progress-text {
            display: flex;
            justify-content: space-between;
            font-size: 0.8rem;
            margin-top: 3px;
        }
        .image-placeholder {
            width: 100%;
            height: 200px;
            background-color: #e9ecef;
            border-radius: 10px;
            margin-top: 10px;
            display: flex;
            align-items: center;
            justify-content: center;
            color: #6c757d;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="chat-container">
            <div class="chat-header">
                <i class="fas fa-robot"></i> DOSTE
            </div>
            <div class="chat-messages" id="chatMessages">
                <div class="message bot-message">
                    <div class="avatar bot-avatar">
                        <i class="fas fa-robot"></i>
                    </div>
                    <div class="message-content">
                        Hi! I'm your smart assistant for tourism and entertainment in Saudi Arabia. How can I help you today?
                    </div>
                </div>
            </div>
            <div class="typing-indicator" id="typingIndicator">
                <div class="typing-dot"></div>
                <div class="typing-dot"></div>
                <div class="typing-dot"></div>
            </div>
            <div class="input-container">
                <form id="chatForm">
                    <div class="input-group">
                        <input type="text" class="form-control" id="userInput" placeholder="Write your Message here ..." required>
                        <button type="submit" class="send-button">
                            <i class="fas fa-paper-plane"></i>
                        </button>
                    </div>
                    <div class="image-option">
                        <input type="checkbox" id="generateImage" class="image-checkbox">
                        <label for="generateImage" class="image-label">Generate image with answer</label>
                    </div>
                </form>
            </div>
        </div>
    </div>

    <script>
        document.getElementById('chatForm').addEventListener('submit', function(e) {
            e.preventDefault();
            const userInput = document.getElementById('userInput');
            const generateImage = document.getElementById('generateImage').checked;
            const message = userInput.value.trim();

            if (message) {
                addMessage(message, 'user');
                showTypingIndicator();
                userInput.value = '';
                google.colab.kernel.invokeFunction('notebook.chat', [message, generateImage], {});
            }
        });

        function addMessage(message, sender, isTranslated = false, imageData = null, imageGenerating = false, messageId = null) {
            const chatMessages = document.getElementById('chatMessages');
            const messageDiv = document.createElement('div');
            messageDiv.className = `message ${sender}-message`;

            if (imageGenerating && messageId) {
                messageDiv.id = messageId;
            }

            const avatar = document.createElement('div');
            avatar.className = `avatar ${sender}-avatar`;
            avatar.innerHTML = sender === 'user' ? '<i class="fas fa-user"></i>' : '<i class="fas fa-robot"></i>';

            const contentWrapper = document.createElement('div');
            const content = document.createElement('div');
            content.className = 'message-content';
            content.textContent = message;
            contentWrapper.appendChild(content);

            if (imageGenerating && sender === 'bot') {
                const imageGeneratingDiv = document.createElement('div');
                imageGeneratingDiv.className = 'image-generating';
                imageGeneratingDiv.innerHTML = '<i class="fas fa-image"></i> جاري توليد الصورة، يرجى الانتظار...';

                const progressContainer = document.createElement('div');
                progressContainer.className = 'progress-container';

                const progressBar = document.createElement('div');
                progressBar.className = 'progress-bar';
                progressBar.id = 'progress-bar-' + messageId;

                const progressText = document.createElement('div');
                progressText.className = 'progress-text';
                progressText.innerHTML = '<span>0%</span><span id="progress-percentage-' + messageId + '">0%</span>';

                progressContainer.appendChild(progressBar);

                const imagePlaceholder = document.createElement('div');
                imagePlaceholder.className = 'image-placeholder';
                imagePlaceholder.id = 'placeholder-' + messageId;
                imagePlaceholder.innerHTML = '<i class="fas fa-image fa-3x"></i>';

                contentWrapper.appendChild(imageGeneratingDiv);
                contentWrapper.appendChild(progressContainer);
                contentWrapper.appendChild(progressText);
                contentWrapper.appendChild(imagePlaceholder);

                startProgressSimulation(messageId);
            }

            if (imageData && sender === 'bot') {
                const image = document.createElement('img');
                image.src = 'data:image/png;base64,' + imageData;
                image.className = 'generated-image';
                image.alt = 'صورة مولدة';
                contentWrapper.appendChild(image);
            }

            if (isTranslated && sender === 'bot') {
                const translationNote = document.createElement('div');
                translationNote.className = 'translation-indicator';
                translationNote.textContent = 'تمت الترجمة من الإنجليزية';
                contentWrapper.appendChild(translationNote);
            }

            messageDiv.appendChild(avatar);
            messageDiv.appendChild(contentWrapper);
            chatMessages.appendChild(messageDiv);
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }

        function showTypingIndicator() {
            document.getElementById('typingIndicator').style.display = 'block';
        }

        function hideTypingIndicator() {
            document.getElementById('typingIndicator').style.display = 'none';
        }

        function startProgressSimulation(messageId) {
            let progress = 0;
            const progressBar = document.getElementById('progress-bar-' + messageId);
            const progressPercentage = document.getElementById('progress-percentage-' + messageId);

            const interval = setInterval(function() {
                progress += 5;
                if (progress >= 100) {
                    clearInterval(interval);
                    return;
                }
                progressBar.style.width = progress + '%';
                progressPercentage.textContent = progress + '%';
            }, 300);
            window['interval-' + messageId] = interval;
        }

        function stopProgressAndShowImage(imageData, messageId) {
            if (window['interval-' + messageId]) {
                clearInterval(window['interval-' + messageId]);
            }

            const messageWithImage = document.getElementById(messageId);
            if (messageWithImage) {
                const placeholder = messageWithImage.querySelector('#placeholder-' + messageId);
                const progressContainer = messageWithImage.querySelector('.progress-container');
                const progressText = messageWithImage.querySelector('.progress-text');
                const imageGeneratingDiv = messageWithImage.querySelector('.image-generating');

                if (placeholder) {
                    if (progressContainer) progressContainer.remove();
                    if (progressText) progressText.remove();
                    if (imageGeneratingDiv) {
                        imageGeneratingDiv.innerHTML = '<i class="fas fa-image"></i> تم توليد الصورة بنجاح!';
                    }

                    const image = document.createElement('img');
                    image.src = 'data:image/png;base64,' + imageData;
                    image.className = 'generated-image';
                    image.alt = 'صورة مولدة';
                    placeholder.parentNode.replaceChild(image, placeholder);
                }
            }
        }

        function handleBotResponse(response, wasTranslated = false, imageData = null, messageId = null) {
            hideTypingIndicator();
            if (document.getElementById('generateImage').checked && !imageData && messageId) {
                addMessage(response, 'bot', wasTranslated, null, true, messageId);
            } else if (imageData && messageId) {
                stopProgressAndShowImage(imageData, messageId);
            } else {
                addMessage(response, 'bot', wasTranslated, imageData);
            }
        }

        function updateImageWhenReady(imageData, messageId) {
            stopProgressAndShowImage(imageData, messageId);
        }
    </script>
</body>
</html>
"""

In [ ]:
from IPython.display import HTML, display
from google.colab import output
import re
import googletrans
from googletrans import Translator
from diffusers import StableDiffusionPipeline
import torch
import os
import base64
from io import BytesIO
from PIL import Image
import time
import logging
import asyncio
import nest_asyncio

# تطبيق nest_asyncio للسماح بتشغيل asyncio داخل حلقة أحداث Colab
nest_asyncio.apply()

# إعداد تسجيل الأخطاء
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# إنشاء كائن المترجم
translator = Translator()

# تحميل نموذج Stable Diffusion مرة واحدة عند بدء التشغيل
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to(device)

# دالة للكشف عن اللغة
def detect_language(text):
    try:
        # تشغيل دالة detect بشكل متزامن باستخدام الحلقة الحالية
        loop = asyncio.get_event_loop()
        detection = loop.run_until_complete(translator.detect(text))
        return detection.lang
    except Exception as e:
        logger.error(f"Language detection error: {e}")
        arabic_pattern = re.compile('[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]+')
        if arabic_pattern.search(text):
            return 'ar'
        return 'en'

# دالة للترجمة غير المتزامنة
async def async_translate_text(text, src, dest):
    try:
        result = await translator.translate(text, src=src, dest=dest)
        return result.text
    except Exception as e:
        logger.error(f"Async translation error: {e}")
        return text  # إرجاع النص الأصلي في حالة فشل الترجمة

# دالة متزامنة للترجمة
def translate_text(text, src, dest):
    try:
        # تشغيل الترجمة غير المتزامنة باستخدام الحلقة الحالية
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(async_translate_text(text, src, dest))
    except Exception as e:
        logger.error(f"Translation error: {e}")
        return text  # إرجاع النص الأصلي في حالة فشل الترجمة

# دالة لتوليد الصور باستخدام النموذج العام
def text_to_image(prompt, height=512, width=512):
    global pipe
    try:
        image = pipe(
            prompt=prompt,
            height=height,
            width=width,
            num_inference_steps=20,
            guidance_scale=7.5,
        ).images[0]

        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        return img_str
    except Exception as e:
        raise Exception(f"فشل في توليد الصورة: {str(e)}")

# دالة للتعامل مع الرسائل
def chat_callback(message, generate_image=False):
    try:
        # كشف اللغة
        detected_lang = detect_language(message)

        # إذا كانت اللغة عربية، نترجمها إلى الإنجليزية
        if detected_lang == 'ar':
            english_message = translate_text(message, 'ar', 'en')
            if english_message == message:  # إذا فشلت الترجمة
                logger.warning("Translation to English failed, using original Arabic message")
                english_message = message  # استخدام النص الأصلي كبديل
        else:
            english_message = message

        # استدعاء دالة generate_answer من النموذج (يفترض أنها تعمل بالإنجليزية)
        try:
            english_response = fine_tuner.generate_answer(
                english_message,
                max_length=256,
                num_return_sequences=1,
                use_history=False
            )
        except Exception as model_error:
            logger.error(f"Model inference error: {model_error}")
            raise Exception("فشل في معالجة الطلب بواسطة النموذج")

        # تحويل الجواب للغة الأصلية التي استخدمها المستخدم
        if detected_lang == 'ar':
            final_response = translate_text(english_response, 'en', 'ar')
            was_translated = True
            if final_response == english_response:  # إذا فشلت الترجمة
                logger.warning("Translation to Arabic failed, using English response")
                final_response = english_response
                was_translated = False
        else:
            final_response = english_response
            was_translated = False

        # تنقية الجواب للتأكد من سلامة تضمينه في JavaScript
        safe_response = final_response.replace('`', '\\`').replace('\n', '\\n')
        unique_id = f'msg-{int(time.time()*1000)}'

        # إرسال الرد النصي أولاً
        if generate_image:
            # إرسال الرد مع إشارة إلى أن الصورة قيد التوليد
            output.eval_js(f'handleBotResponse(`{safe_response}`, {str(was_translated).lower()}, null, "{unique_id}")')

            # توليد الصورة بناءً على الإجابة باللغة الإنجليزية
            try:
                img_prompt = english_response
                img_data = text_to_image(img_prompt)
                output.eval_js(f'updateImageWhenReady(`{img_data}`, "{unique_id}")')
            except Exception as img_error:
                error_message = f"فشل توليد الصورة: {str(img_error)}"
                output.eval_js(f'handleBotResponse(`{error_message}`, false, null)')
        else:
            # إرسال الرد بدون صورة
            output.eval_js(f'handleBotResponse(`{safe_response}`, {str(was_translated).lower()}, null)')

    except Exception as e:
        logger.error(f"General error in chat_callback: {e}")
        error_message = f"عذراً، حدث خطأ: {str(e)}"
        output.eval_js(f'handleBotResponse(`{error_message}`, false, null)')

# تسجيل الدالة كـ callback
output.register_callback('notebook.chat', chat_callback)

# عرض الواجهة
display(HTML(html))

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['during the cooler months , when the weather is cooler .']


  0%|          | 0/20 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['the 1 st century .']


  0%|          | 0/20 [00:00<?, ?it/s]